In [95]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [96]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [97]:
#
n='Input_A2_016'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_A2_016
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.008788277629346544
test_rmse: 0.009079627197977655
test_r2: 0.5088963969039269
feature number: 3
train_rmse: 0.008477487835452484
test_rmse: 0.008808154790360715
test_r2: 0.5365004908227714
feature number: 4
train_rmse: 0.008181302740127188
test_rmse: 0.008509857811724052
test_r2: 0.5690125990799022
feature number: 5
train_rmse: 0.008148505427006467
test_rmse: 0.008503435128398993
test_r2: 0.5697894905455707
feature number: 6
train_rmse: 0.008044798708574738
test_rmse: 0.00842174415434341
test_r2: 0.5779946965784081
feature number: 7
train_rmse: 0.008037700766365799
test_rmse: 0.008429081780223977
test_r2: 0.577175770795938
feature number: 8
train_rmse: 0.008028864729087521
test_rmse: 0.008428214455153961
test_r2: 0.5771949852630488
feature number: 9
train_rmse: 0.008021166376215437
test_rmse: 0.008470190176667615
test_r2: 0.5733351515250095
feature number: 10
train_rmse: 0.007989596729066649
test_rmse: 0.0085491047261

train_rmse: 0.00791822831167259
test_rmse: 0.008778672328997917
test_r2: 0.5420149206170051
feature number: 17
train_rmse: 0.007892347393384336
test_rmse: 0.008761221554147675
test_r2: 0.5440381773611984
feature number: 18
train_rmse: 0.00788675741999125
test_rmse: 0.00879187151129859
test_r2: 0.540724611536539
feature number: 19
train_rmse: 0.007786768510681451
test_rmse: 0.008713815546174597
test_r2: 0.5486525510165772
feature number: 20
train_rmse: 0.007762302833102685
test_rmse: 0.008731681503967513
test_r2: 0.5471833715195537
Test_rmse_min = 1.fea_num: 6  2.rmse: 0.00842174415434341
Test_r2_max = 1. fea_num: 6  2.r2: 0.5779946965784081
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.008788277629346544
test_rmse: 0.009079627197977655
test_r2: 0.5088963969039269
feature number: 3
train_rmse: 0.008477487835452484
test_rmse: 0.008808154790360715
test_r2: 0.5365004908227714
feature number: 4
train_rmse: 0.008181302740127188
test_rmse: 0.008509857811724052
test_r2: 0.569012599079

train_rmse: 0.008021166376215437
test_rmse: 0.008470190176667615
test_r2: 0.5733351515250095
feature number: 10
train_rmse: 0.007989596729066649
test_rmse: 0.008549104726173208
test_r2: 0.5650616362471967
feature number: 11
train_rmse: 0.007981193744971467
test_rmse: 0.008581680782726836
test_r2: 0.5618494272461202
feature number: 12
train_rmse: 0.007967289423163753
test_rmse: 0.008581704819460625
test_r2: 0.5618772374429634
feature number: 13
train_rmse: 0.007957278326445903
test_rmse: 0.008604076313499506
test_r2: 0.559924676284714
feature number: 14
train_rmse: 0.007942331126945626
test_rmse: 0.00868762921178079
test_r2: 0.5510265006399556
feature number: 15
train_rmse: 0.00793544227215513
test_rmse: 0.008704434718082557
test_r2: 0.5493185688147578
feature number: 16
train_rmse: 0.00791822831167259
test_rmse: 0.008778672328997917
test_r2: 0.5420149206170051
feature number: 17
train_rmse: 0.007892347393384336
test_rmse: 0.008761221554147675
test_r2: 0.5440381773611984
feature number:

In [117]:
fea_number=6
#
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
#
y=data.Predict
skew_data=skew_pro(data)

num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_019', 'Input_A2_018', 'Input_A4_016', 'Input_A6_017',
       'Input_A6_016', 'Input_A5_019'], dtype=object)

## Linear Regression Baseline

In [112]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [113]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.008044798708574738
test_rmse: 0.00842174415434341
test_r2: 0.5779946965784081


## Lasso

In [114]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [118]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009]}
laso_alpha(skew_x,y,parameters)

[{'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.001}, {'alpha': 0.0009}, {'alpha': 0.001}, {'alpha': 0.0009}, {'alpha': 0.0009}, {'alpha': 0.0009}]


In [153]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 9e-06}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-06}, {'alpha': 9e-05}, {'alpha': 9e-06}, {'alpha': 9e-05}]


In [119]:
laso_rmse_ave(skew_x,y,0.0009)

test_rmse_ave: 0.0086226481913232
[0.008627348767481044, 0.008254346319483786, 0.008972650320135209, 0.008337849503730899, 0.007897738561987735, 0.007829492170014966, 0.009301224693746113, 0.009236384340745606, 0.008655464361822332, 0.009113982874084306]


test_r2_ave: 0.5577594313777336
[0.6175623896998085, 0.633543736913505, 0.48865962061539725, 0.5815632677301809, 0.6388179027152858, 0.6447738416314625, 0.4245776063066312, 0.49756964586844377, 0.5290729621591038, 0.5214533401375185]


In [154]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.008409586376601795
[0.008075465298330036, 0.00791152396369169, 0.008917094790309695, 0.008271623607208066, 0.007606711338365201, 0.007526075286801399, 0.009575456599207865, 0.009101761342332816, 0.00821787984201634, 0.008892271697754828]


test_r2_ave: 0.5770345886918602
[0.6649257948260741, 0.6633511990408596, 0.4949721011252771, 0.5881839904552834, 0.6649461746399874, 0.6717725733826536, 0.39014656858667873, 0.5121090481994248, 0.5754855726766388, 0.5444528639857233]


## ElasticNet

In [120]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [122]:
alphas = [0.04]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02]
l1ratio = [0.02,0.01,0.03,0.04]
ElasticNet_alpha(skew_x,y,alphas,l1ratio)

alpha
[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]


l1_ratio
[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]


In [156]:

alphas = [0.02]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02]
l1ratio = [0.02,0.01,0.03,0.04]
ElasticNet_alpha(,y,alphas,l1ratio)

alpha
[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [158]:
ElasticNet_rmse_ave(skew_data[title],y,[0.02],0.01)

test_rmse_ave: 0.00839153662939823
[0.00812996697614208, 0.007878619251568257, 0.008766092393325976, 0.008306545835593546, 0.007634577632503669, 0.007580113089361061, 0.009502217942602595, 0.009062295119254738, 0.008163387894411352, 0.008891550159219011]


test_r2_ave: 0.5794087765334078
[0.660387670622838, 0.6661456786234328, 0.5119315989675998, 0.5846993319781477, 0.6624868173694487, 0.6670422554959089, 0.39943991856589856, 0.5163309709842439, 0.5810967335417028, 0.5445267891848572]


In [123]:
ElasticNet_rmse_ave(skew_x,y,[0.04],0.02)

test_rmse_ave: 0.008603923529016276
[0.008691264493328311, 0.008211822611262246, 0.008869690856676703, 0.008291483693613865, 0.007887049185547366, 0.007957257216058099, 0.009165033751095656, 0.00923311734101793, 0.008690623273789791, 0.009041892867772797]


test_r2_ave: 0.5602060116330803
[0.6118748200894426, 0.6373097382726307, 0.5003273598363914, 0.5862040841122467, 0.6397949415521604, 0.6330857792376535, 0.4413052058407685, 0.4979250120968066, 0.5252393352591682, 0.5289938400335337]


## SVR

In [39]:
from sklearn.svm import SVR

In [129]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear', 'rbf','poly'], 'C':[1.2,1.3,1.1],'gamma': [1e-7,5e-6,5e-7],'epsilon':[0.2,0.1,0.05,0]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.1, 'epsilon': 0, 'gamma': 1e-07, 'kernel': 'linear'}




In [126]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6,1e-8,5e-7],'epsilon':[0.1,0.05,0,0.2]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}


best params
{'C': 1.5, 'epsilon': 0, 'gamma': 1e-06, 'kernel': 'rbf'}




In [72]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    y = data['Predict']
    if fea_num != None:
        num=data.select_dtypes(exclude='object')
        numcorr=num.corr()
        title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
        X= data[title]
    else:
        X= data.drop(['Predict'], axis=1)
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [130]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,6,model)

train_rmse: 0.008447081811618556
test_rmse: 0.008784714647108189
test_r2: 0.5381124015587389


In [127]:
model=SVR(kernel='rbf', C=1.5, gamma= 1e-06, epsilon= 0)
svr_rmse_ave(skew_data,None,model)

train_rmse: 0.010811210684863716
test_rmse: 0.010947800915324083
test_r2: 0.29369844273506174


## XGB

In [44]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [46]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[04:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 70}


[04:02:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

In [133]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['gbtree'],
    'n_estimators':range(60,161,10),
#     'n_estimators':range(150,230,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[12:03:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[12:03:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 100}


[12:03:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 140}


[12:03:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 100}


[12:03:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 110}


[12:04:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booste

### 調 max_depth / min_child_weight

In [48]:
# {'booster': 'dart', 'n_estimators': 60}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:04:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 4}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

In [138]:
# {'booster': 'gbtree', 'n_estimators': 100}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [5],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:25:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 1}


[12:25:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 1}


[12:25:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 1}


[12:25:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 4}


[12:25:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 3}


[12:25:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_

### 調 gamma

In [49]:
# {'max_depth': 2, 'min_child_weight': 2}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:05:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



In [139]:
# {'max_depth': 5, 'min_child_weight': 3}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 3, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:26:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:26:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:26:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:27:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:27:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:27:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[12:27:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



### 調 subsample / colsample_bytree

In [51]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.3}


[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.9}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

In [141]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.65],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 3, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:35:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.65, 'subsample': 0.7}


[12:35:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.65, 'subsample': 0.5}


[12:35:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.65, 'subsample': 0.4}


[12:36:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.65, 'subsample': 0.9}


[12:36:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.65, 'subsample': 0.7}


[12:36:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best para

### reg_alpha / reg_lambda

In [52]:
# {'colsample_bytree': 0.5, 'subsample': 0.6}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:10:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:27]

In [143]:
# {'colsample_bytree': 0.65, 'subsample': 0.7}

parameters_for_testing = {
#     'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
    'reg_alpha': [0.02],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 3, 'seed': 42,
 'gamma':0,'subsample':0.7,'colsample_bytree': 0.65}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:41:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 2}


[12:41:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 3}


[12:41:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 1}


[12:42:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 2}


[12:42:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 2}


[12:42:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.02, 'reg_lambda': 0.1

### learning rate

In [53]:
# {'reg_alpha': 0, 'reg_lambda': 0.7}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5,'reg_alpha': 0, 'reg_lambda': 0.7}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [144]:
# {'reg_alpha': 0.02, 'reg_lambda': 1.5}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'gbtree', 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 3, 'seed': 42,
 'gamma':0,'subsample':0.7,'colsample_bytree': 0.65,'reg_alpha': 0.02, 'reg_lambda': 1.5}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[12:44:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[12:44:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[12:44:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:44:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:44:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:44:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[12:44:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depre

In [131]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 60, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008496031540705907
[0.007648650629856374, 0.007861428699795805, 0.008923526006532567, 0.008422016710058338, 0.007514787893473913, 0.007671313900216453, 0.009908224384169528, 0.0094945394022681, 0.008777706642056635, 0.008738121138631347]


test_r2_ave: 0.5658204873179591
[0.6994093043442213, 0.6676009790074625, 0.49424336268772506, 0.5730727307503245, 0.6729951732468282, 0.6589820348516682, 0.3470225948052349, 0.46909149956225205, 0.5156770986673376, 0.5601100952565365]


In [145]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)
xgb_ave(skew_x, y,best_xgb_model)

test_rmse_ave: 0.00880764711951124
[0.008162968029984506, 0.008577762353002276, 0.009462903638912055, 0.00855081208142069, 0.007939599625002633, 0.008189740832580842, 0.009093054026034867, 0.009414726896857857, 0.009560799731481136, 0.009124103979835523]


test_r2_ave: 0.5373193386579707
[0.6576249751286939, 0.6042646958273833, 0.4312552000067853, 0.5599151457576778, 0.6349789465584614, 0.6113326421592595, 0.4500464240450244, 0.47797977488315724, 0.42540568727536754, 0.5203898949378959]


## RandomForest

In [77]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [78]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [79]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [81]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [20,40,50,60,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0688s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  64 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 108 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 134 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 190 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 220 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 254 tasks      | elapsed:    9.5s
[Parallel(n_jobs=6)]: Done 288 tasks      | elapse

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50}
best score
-6.733198261475197e-05


[Parallel(n_jobs=6)]: Done 1800 out of 1800 | elapsed:  1.4min finished


In [82]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    7.6s
[Parallel(

best params
{'max_leaf_nodes': 14, 'min_samples_leaf': 2}
best score
-6.728825706399575e-05


[Parallel(n_jobs=6)]: Done 585 out of 585 | elapsed:   24.0s finished


In [83]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 14,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.008359151931886303
[0.008046421179911343, 0.007522207654460264, 0.008970754200764595, 0.008192033878800358, 0.007261528804827015, 0.007389966886686526, 0.009434354784131698, 0.009095030814723662, 0.00923126977742949, 0.008447951337128053]


test_r2_ave: 0.5800138319443187
[0.6673317079609893, 0.6956681454215288, 0.4888757128331376, 0.5960708670050614, 0.6946648253743058, 0.6835371494626516, 0.40798747419915105, 0.5128303480153389, 0.46433192124209743, 0.5888401679289246]


In [146]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_x, y, test_size=0.2, random_state=42)

In [149]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [50,60,70],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    4.3s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    6.8s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    7.7s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    9.1s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   10.4s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   12.2s
[Parallel(

best params
{'max_depth': 7, 'min_samples_split': 5, 'n_estimators': 50}
best score
-7.248795889365611e-05


In [150]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.3s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.6s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    7.1s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    8.5s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    9.9s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   11.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   13.3s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   15.3s
[Parallel(

best params
{'max_leaf_nodes': 13, 'min_samples_leaf': 2}
best score
-7.187565556136015e-05


In [151]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            max_leaf_nodes = 13,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_x, y,rand_model)

test_rmse_ave: 0.008571425994027312
[0.008348141516689295, 0.008700447155743007, 0.0090152328313417, 0.007994644048066483, 0.0073991999818512596, 0.007588382588342307, 0.00919271720586972, 0.009217181079467342, 0.009288701921338313, 0.008969611611563701]


test_r2_ave: 0.5614890289050415
[0.6419155262763883, 0.5928636082696312, 0.4837946532567807, 0.6153019701888609, 0.6829773880705825, 0.6663153809494926, 0.4379249758667355, 0.49965666848948576, 0.45764589267399014, 0.5364942250084674]


In [ ]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [89]:
model=rand_model
X_train, X_test, y_train, y_test = train_test_split(skew_data, y, test_size=0.2, random_state=0)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(math.sqrt(mean_squared_error(y_test, y_pred )))
print(r2_score(y_test, y_pred ))

8.232490110738641e-18
1.0


In [90]:
y_test

144   -0.05
6     -0.04
61    -0.03
342   -0.02
55    -0.03
       ... 
234   -0.05
302   -0.01
21    -0.05
251   -0.05
133   -0.02
Name: Predict, Length: 69, dtype: float64

In [94]:
y

0     -0.02
1     -0.03
2     -0.05
3     -0.05
4     -0.04
       ... 
343   -0.01
344   -0.02
345   -0.01
346   -0.02
347   -0.01
Name: Predict, Length: 345, dtype: float64